## Wealth Of Nations Dashboard
1. Use the dropdown to select the data type
2. Use the slider to show the heatmap for a specific year
3. Use the animation button to animate the choropleth for all the years
4. Clicking on any country toggles the display of timeseries for that country. (Clicking on oceans un-selects all the countries)

In [ ]:
import numpy as np
import pandas as pd

import ipywidgets as widgets
import bqplot as bq
import bqplot.pyplot as plt

In [ ]:
initial_year = 1800

In [ ]:
data = pd.read_json("nations.json")
country_codes = pd.read_csv("world_map_codes.csv")
country_code_map = dict(zip(country_codes["ISON3"], country_codes["Name"]))

In [ ]:
country_data = (
    data.merge(country_codes, left_on="name", right_on="Name")[
        ["name", "ISON3", "region", "income", "lifeExpectancy", "population"]
    ]
    .rename(columns={"ISON3": "id"})
    .set_index("id")
)

In [ ]:
def get_time_series(col):
    time_series_df = country_data[col].apply(dict).apply(pd.Series)
    time_series_df.index.name = None
    time_series_df.columns = time_series_df.columns.astype("int")

    return time_series_df

In [ ]:
income_time_series = get_time_series("income")
life_exp_time_series = get_time_series("lifeExpectancy")
pop_time_series = get_time_series("population")

In [ ]:
income_min, income_max = income_time_series.min().min(), income_time_series.max().max()
life_exp_min, life_exp_max = (
    life_exp_time_series.min().min(),
    life_exp_time_series.max().max(),
)
pop_min, pop_max = pop_time_series.min().min(), pop_time_series.max().max()

In [ ]:
data_dict = {
    "Income": income_time_series,
    "Population": pop_time_series,
    "Life Expectancy": life_exp_time_series,
}

min_max_dict = {
    "Income": (np.log(income_min), np.log(income_max)),
    "Population": (np.log(pop_min), np.log(pop_max)),
    "Life Expectancy": (life_exp_min, life_exp_max),
}

color_scheme_dict = {
    "Income": "Greens",
    "Population": "Blues",
    "Life Expectancy": "Oranges",
}

data_type_dropdown = widgets.Dropdown(
    description="Data Type",
    options=["Income", "Life Expectancy", "Population"],
    value="Population",
)

year_slider = widgets.IntSlider(
    description="year",
    min=1800,
    max=2008,
    continuous_update=False,
    layout=widgets.Layout(width="300px"),
)

play_button = widgets.Play(min=1800, max=2008, interval=10)
widgets.jslink((play_button, "value"), (year_slider, "value"))

# world map
world_map_fig_title_tmpl = "World {data_type} for year {year}"
world_map_fig = plt.figure(
    layout=widgets.Layout(width="1000px", height="500px"),
    fig_margin=dict(top=45, bottom=10, left=20, right=20),
)
tooltip = bq.Tooltip(fields=["name"], labels=["Country"])

plt.scales(scales={"projection": bq.Mercator(), "color": bq.ColorScale()})

world_map = plt.geo(
    map_data="WorldMap",
    tooltip=tooltip,
    interactions={"click": "select", "hover": "tooltip"},
    axes_options={"color": {"visible": False}},
    selected_styles={"selected_fill": "yellow"},
    colors={"default_color": "Grey"},
)

# data time series fig
ts_fig_title_tmpl = "{data_type} Time Series"
time_series_fig = plt.figure(
    layout=widgets.Layout(width="1000px", height="400px"),
    fig_margin=dict(top=40, bottom=30, left=60, right=40),
)
country_time_series = plt.plot(
    list(pop_time_series.columns),
    [],
    axes_options={"x": {"label": "Year"}, "y": {"tick_format": "~s"}},
    display_legend=True,
)

# link widgets
def on_data_type_update(*args):
    global data_time_series
    # reset color and selected attributes of world_map
    world_map.color = {}
    world_map.selected = []

    data_type = data_type_dropdown.value
    data_time_series = data_dict[data_type]

    scale_min, scale_max = min_max_dict[data_type]
    color_scale = world_map.scales["color"]
    color_scale.scheme = color_scheme_dict[data_type]
    color_scale.min = scale_min
    color_scale.max = scale_max
    year_slider.value = initial_year


data_type_dropdown.observe(on_data_type_update, "value")


def update_color(*args):
    data_type = data_type_dropdown.value
    year = year_slider.value

    if year in data_time_series.columns:
        if data_type in ["Population", "Income"]:
            color_data = np.log(data_time_series[year])
        else:
            color_data = data_time_series[year]
        world_map.color = dict(color_data)
    else:
        world_map.color = {}

    world_map_fig.title = world_map_fig_title_tmpl.format(
        data_type=data_type, year=year
    )


year_slider.observe(update_color, "value")


def plot_time_series_for_country(*args):
    if world_map.selected is not None and len(world_map.selected) > 0:
        selected_countries = world_map.selected
        country_time_series.y = data_time_series.loc[selected_countries]
        country_time_series.labels = [
            country_code_map[c_id] for c_id in selected_countries
        ]
    else:
        country_time_series.labels = []
        country_time_series.y = []
    time_series_fig.title = ts_fig_title_tmpl.format(data_type=data_type_dropdown.value)


world_map.observe(plot_time_series_for_country, "selected")

on_data_type_update(None)

widgets.VBox(
    [
        world_map_fig,
        widgets.HBox(
            [
                time_series_fig,
                widgets.VBox([data_type_dropdown, year_slider, play_button]),
            ]
        ),
    ]
)